In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from pathlib import Path

# Import Rust bindings
try:
    import hft_py.portfolio_drift as pdrift
    print("✓ Successfully imported portfolio drift uncertainty module")
except ImportError as e:
    print(f"❌ Could not import hft_py.portfolio_drift: {e}")
    print("Please rebuild the Rust bindings: cd rust_python_bindings && maturin develop --release")
    sys.exit(1)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
np.random.seed(42)

## Setup: Generate Sample Market Data

We'll create a simple 3-asset universe with:
- Different expected returns
- Correlated volatilities
- Uncertain drift parameters

In [ ]:
# Market parameters
n_assets = 3
asset_names = ["Stock A", "Stock B", "Stock C"]

# Expected returns (annualized)
expected_returns = np.array([0.10, 0.12, 0.08])  # 10%, 12%, 8%

# Covariance matrix (annualized)
volatilities = np.array([0.20, 0.25, 0.15])  # 20%, 25%, 15%
correlation = np.array([
    [1.0, 0.5, 0.3],
    [0.5, 1.0, 0.4],
    [0.3, 0.4, 1.0]
])

# Construct covariance matrix: Σ = D * C * D where D = diag(σ)
D = np.diag(volatilities)
covariance = D @ correlation @ D

print("Expected Returns:")
for i, name in enumerate(asset_names):
    print(f"  {name}: {expected_returns[i]:.1%}")

print("\nVolatilities:")
for i, name in enumerate(asset_names):
    print(f"  {name}: {volatilities[i]:.1%}")

print("\nCovariance Matrix:")
print(pd.DataFrame(covariance, index=asset_names, columns=asset_names).round(4))

## Example 1: Robust Portfolio Choice

We solve the **worst-case portfolio optimization** problem:

$$\max_q \min_{\mu \in [\hat{\mu} - \delta, \hat{\mu} + \delta]} E[U(wealth)]$$

where:
- $q$ are portfolio weights
- $\hat{\mu}$ is the estimated drift
- $\delta$ is the drift uncertainty
- $U$ is CARA utility: $U(W) = -\exp(-\gamma W)$

In [ ]:
# Portfolio optimization parameters
risk_aversion = 2.0  # γ = 2 (moderate risk aversion)
drift_uncertainty_levels = [0.0, 0.01, 0.02, 0.05]  # 0%, 1%, 2%, 5%

results = []

for delta in drift_uncertainty_levels:
    result = pdrift.portfolio_choice_drift_uncertainty(
        expected_returns=expected_returns.tolist(),
        covariance=covariance.flatten().tolist(),
        risk_aversion=risk_aversion,
        drift_uncertainty=delta
    )
    
    results.append({
        'drift_uncertainty': delta,
        'weights': result.weights,
        'expected_return': result.expected_return,
        'worst_case_return': result.worst_case_return,
        'variance': result.variance,
        'utility': result.utility
    })
    
    print(f"\n=== Drift Uncertainty: {delta:.1%} ===")
    print(f"Optimal Weights:")
    for i, (name, w) in enumerate(zip(asset_names, result.weights)):
        print(f"  {name}: {w:>8.3f} ({w*100:>6.2f}%)")
    print(f"Expected Return:    {result.expected_return:.4f}")
    print(f"Worst-Case Return:  {result.worst_case_return:.4f}")
    print(f"Portfolio Variance: {result.variance:.6f}")
    print(f"CARA Utility:       {result.utility:.6f}")

In [ ]:
# Visualize how portfolio weights change with uncertainty
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Portfolio weights vs uncertainty
for i, name in enumerate(asset_names):
    weights = [r['weights'][i] for r in results]
    ax1.plot([r['drift_uncertainty'] for r in results], weights, 
             marker='o', label=name, linewidth=2)

ax1.set_xlabel('Drift Uncertainty', fontsize=12)
ax1.set_ylabel('Portfolio Weight', fontsize=12)
ax1.set_title('Portfolio Allocation vs Drift Uncertainty', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Expected vs Worst-Case Returns
ax2.plot([r['drift_uncertainty'] for r in results], 
         [r['expected_return'] for r in results],
         marker='o', label='Expected Return', linewidth=2)
ax2.plot([r['drift_uncertainty'] for r in results], 
         [r['worst_case_return'] for r in results],
         marker='s', label='Worst-Case Return', linewidth=2, linestyle='--')

ax2.set_xlabel('Drift Uncertainty', fontsize=12)
ax2.set_ylabel('Return', fontsize=12)
ax2.set_title('Return vs Drift Uncertainty', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Example 2: Optimal Liquidation Strategy

Suppose we need to liquidate a large position over time. The optimal strategy balances:
- **Market risk**: Holding the position exposes us to price movements
- **Price impact**: Trading too fast causes adverse price impact
- **Drift uncertainty**: We don't know the true expected return

The solution is an **exponential decay** trading schedule.

In [ ]:
# Liquidation parameters
initial_position = 1000.0  # shares to liquidate
time_horizon = 1.0  # 1 day
n_steps = 50  # 50 time steps

# Market microstructure parameters
drift = 0.0  # No expected drift
volatility = 0.02  # 2% daily volatility
temporary_impact = 0.001  # Temporary impact coefficient
permanent_impact = 0.0001  # Permanent impact coefficient
risk_aversion = 1.0

# Compare different drift uncertainty levels
uncertainty_levels = [0.0, 0.005, 0.01, 0.02]
liq_results = {}

for delta in uncertainty_levels:
    result = pdrift.liquidation_drift_uncertainty(
        initial_position=initial_position,
        time_horizon=time_horizon,
        n_steps=n_steps,
        drift=drift,
        drift_uncertainty=delta,
        volatility=volatility,
        temporary_impact=temporary_impact,
        permanent_impact=permanent_impact,
        risk_aversion=risk_aversion
    )
    liq_results[delta] = result
    
    print(f"\nDrift Uncertainty: {delta:.1%}")
    print(f"  Expected Cost:    ${result.expected_cost:.2f}")
    print(f"  Worst-Case Cost:  ${result.worst_case_cost:.2f}")
    print(f"  Final Position:    {result.trading_schedule[-1]:.2f} shares")

In [ ]:
# Visualize liquidation schedules
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Trading schedules
for delta, result in liq_results.items():
    ax1.plot(result.times, result.trading_schedule, 
             label=f'δ = {delta:.1%}', linewidth=2)

ax1.set_xlabel('Time (days)', fontsize=12)
ax1.set_ylabel('Position (shares)', fontsize=12)
ax1.set_title('Optimal Liquidation Schedules', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Trading rates (velocity)
for delta, result in liq_results.items():
    times = result.times[:-1]  # Exclude last point
    ax2.plot(times, np.abs(result.trading_rates), 
             label=f'δ = {delta:.1%}', linewidth=2)

ax2.set_xlabel('Time (days)', fontsize=12)
ax2.set_ylabel('|Trading Rate| (shares/day)', fontsize=12)
ax2.set_title('Trading Velocity', fontsize=14, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("• Higher drift uncertainty → slower liquidation (more caution)")
print("• Exponential decay minimizes worst-case cost")
print("• Trading rate is highest initially, then decreases")

## Example 3: Portfolio Transition (Rebalancing)

Now suppose we want to transition from our **current portfolio** to a **target portfolio**.

The optimization accounts for:
- Transaction costs
- Portfolio risk during transition
- Drift uncertainty

In [ ]:
# Current and target portfolios
initial_weights = np.array([0.5, 0.3, 0.2])  # Current allocation
target_weights = np.array([0.3, 0.4, 0.3])   # Desired allocation

print("Initial Portfolio:")
for name, w in zip(asset_names, initial_weights):
    print(f"  {name}: {w:.1%}")

print("\nTarget Portfolio:")
for name, w in zip(asset_names, target_weights):
    print(f"  {name}: {w:.1%}")

# Transition parameters
time_horizon = 1.0  # 1 day transition
n_steps = 20
transaction_cost = 0.001  # 0.1% per trade
drift_uncertainty = 0.02
risk_aversion = 2.0

transition_result = pdrift.transition_drift_uncertainty(
    initial_weights=initial_weights.tolist(),
    target_weights=target_weights.tolist(),
    time_horizon=time_horizon,
    n_steps=n_steps,
    expected_returns=expected_returns.tolist(),
    covariance=covariance.flatten().tolist(),
    drift_uncertainty=drift_uncertainty,
    transaction_cost=transaction_cost,
    risk_aversion=risk_aversion
)

print(f"\n=== Transition Results ===")
print(f"Expected Cost:    ${transition_result.expected_cost:.4f}")
print(f"Worst-Case Cost:  ${transition_result.worst_case_cost:.4f}")

print("\nFinal Weights:")
final_weights = transition_result.trajectory[-1]
for name, w in zip(asset_names, final_weights):
    print(f"  {name}: {w:.1%}")

In [ ]:
# Visualize the transition
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot 1: Portfolio weights over time
times = transition_result.times
trajectory = np.array(transition_result.trajectory)

for i, name in enumerate(asset_names):
    ax1.plot(times, trajectory[:, i], label=name, linewidth=2, marker='o', markersize=3)
    # Add target line
    ax1.axhline(y=target_weights[i], color=f'C{i}', linestyle='--', alpha=0.5)

ax1.set_xlabel('Time (days)', fontsize=12)
ax1.set_ylabel('Portfolio Weight', fontsize=12)
ax1.set_title('Portfolio Transition Path', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Trading rates (changes in weights)
trading_rates = np.array(transition_result.trading_rates)
times_rates = times[:-1]  # One fewer point

for i, name in enumerate(asset_names):
    ax2.plot(times_rates, trading_rates[:, i], label=name, linewidth=2, marker='s', markersize=3)

ax2.set_xlabel('Time (days)', fontsize=12)
ax2.set_ylabel('Trading Rate (dw/dt)', fontsize=12)
ax2.set_title('Trading Velocity During Transition', fontsize=14, fontweight='bold')
ax2.legend()
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key Insights:")
print("• Exponential convergence to target weights")
print("• Trading intensity decreases over time")
print("• Balances transaction costs vs. tracking error")

## Example 4: Risk Measures Under Drift Uncertainty

Let's compute Value-at-Risk (VaR) and Expected Shortfall (CVaR) accounting for drift uncertainty.

In [ ]:
# Portfolio to analyze
portfolio_weights = np.array([0.4, 0.35, 0.25])
confidence_level = 0.95  # 95% confidence
time_horizon = 1.0 / 252  # 1 trading day

print(f"Portfolio:")
for name, w in zip(asset_names, portfolio_weights):
    print(f"  {name}: {w:.1%}")

print(f"\nRisk Measures (1-day horizon, {confidence_level:.0%} confidence):")

for delta in [0.0, 0.01, 0.02, 0.05]:
    var = pdrift.var_drift_uncertainty(
        weights=portfolio_weights.tolist(),
        expected_returns=expected_returns.tolist(),
        covariance=covariance.flatten().tolist(),
        drift_uncertainty=delta,
        confidence_level=confidence_level,
        time_horizon=time_horizon
    )
    
    cvar = pdrift.cvar_drift_uncertainty(
        weights=portfolio_weights.tolist(),
        expected_returns=expected_returns.tolist(),
        covariance=covariance.flatten().tolist(),
        drift_uncertainty=delta,
        confidence_level=confidence_level,
        time_horizon=time_horizon
    )
    
    print(f"\n  Drift Uncertainty: {delta:.1%}")
    print(f"    VaR:   {var*100:.3f}%")
    print(f"    CVaR:  {cvar*100:.3f}%")

In [ ]:
# Visualize risk measures vs uncertainty
uncertainty_range = np.linspace(0, 0.05, 20)
vars = []
cvars = []

for delta in uncertainty_range:
    var = pdrift.var_drift_uncertainty(
        weights=portfolio_weights.tolist(),
        expected_returns=expected_returns.tolist(),
        covariance=covariance.flatten().tolist(),
        drift_uncertainty=delta,
        confidence_level=confidence_level,
        time_horizon=time_horizon
    )
    
    cvar = pdrift.cvar_drift_uncertainty(
        weights=portfolio_weights.tolist(),
        expected_returns=expected_returns.tolist(),
        covariance=covariance.flatten().tolist(),
        drift_uncertainty=delta,
        confidence_level=confidence_level,
        time_horizon=time_horizon
    )
    
    vars.append(var * 100)  # Convert to percentage
    cvars.append(cvar * 100)

plt.figure(figsize=(10, 6))
plt.plot(uncertainty_range * 100, vars, label='VaR (95%)', linewidth=2, marker='o')
plt.plot(uncertainty_range * 100, cvars, label='CVaR (95%)', linewidth=2, marker='s')
plt.xlabel('Drift Uncertainty (%)', fontsize=12)
plt.ylabel('Risk Measure (%)', fontsize=12)
plt.title('Risk Measures vs Drift Uncertainty', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\n💡 Observations:")
print("• Both VaR and CVaR increase linearly with drift uncertainty")
print("• CVaR > VaR (expected loss given breach)")
print("• Drift uncertainty significantly impacts tail risk")

## Summary

This notebook demonstrated the implementation of portfolio optimization under drift uncertainty:

### Key Takeaways:

1. **Robust Portfolio Choice**:
   - Optimizes worst-case utility when drift is uncertain
   - Conservative allocation compared to standard Markowitz
   - Weights adapt to uncertainty level

2. **Optimal Liquidation**:
   - Exponential decay schedule minimizes worst-case cost
   - Balances market risk vs. price impact
   - Higher uncertainty → slower trading

3. **Portfolio Transition**:
   - Smooth rebalancing path to target weights
   - Accounts for transaction costs
   - Exponential convergence

4. **Risk Measures**:
   - VaR and CVaR under drift uncertainty
   - Tail risk increases with uncertainty
   - Important for risk management

### Further Reading:

- Original paper: Bismuth, A., Guéant, O., & Pu, J. "Portfolio choice, portfolio liquidation, and portfolio transition under drift uncertainty"
- Related: Robust portfolio optimization, ambiguity aversion, market microstructure